## Spark SQL v jednoduchých příkladech

Více k jednotlivým metodám viz [Manual Spark SQL](http://spark.apache.org/docs/1.6.0/api/python/pyspark.sql.html)

### Spuštění PySpark

`pyspark --master yarn --num-executors 2 --executor-memory 4G --packages com.databricks:spark-csv_2.10:1.5.0 --conf spark.ui.port=1<ddmm>`, kde `<ddmm>` je váš den a měsíc narození, napr. spark.ui.port=10811

In [ ]:
# uzitecny import
from pyspark.sql import functions as F

In [ ]:
### nacteni DataFrame z databaze Hive a nakesovani
Tep_DF = sqlContext.sql('select * from fel_bigdata.teplota').cache()

### Základní informace o DataFrame

In [ ]:
Tep_DF.show() # vypis ukazky dat jako DataFrame
Tep_DF.take(5) # vypis ukazky dat jako RDD

Tep_DF.count() # pocet radku

Tep_DF.printSchema() # schema - nazvy sloupcu a typy

### Výběr sloupců a řádků

In [ ]:
### vyber sloupcu
Tep_DF2 = Tep_DF.select('stat', 'mesic', 'teplota')
Tep_DF2.show()

### vyber radku - ruzne zpusoby
Tep_DF2 = Tep_DF.filter(Tep_DF['stat']=='TX')
Tep_DF2.show()

Tep_DF2 = Tep_DF.filter((Tep_DF['stat']=='TX') & (Tep_DF['mesic']==11)) # zavorky jsou nutne
Tep_DF2.show()

# to same, ale podminka v jine syntaxi
Tep_DF2 = Tep_DF.filter('stat="TX"')
Tep_DF2.show()

Tep_DF2 = Tep_DF.filter('stat="TX" and mesic=11')
Tep_DF2.show()

# RDD syntaxe
Tep_DF.filter(lambda r: r[9]=='TX' and r[1]==11).take(5) # nefunguje, ocekava syntaxi pro DataFrame  
Tep_DF.rdd.filter(lambda r: r[9]=='TX' and r[1]==11).take(5) # po explicitnim prevodu na RDD funguje 

### pouze radky bez duplicit (unikatni hodnoty)
Tep_DF2 = Tep_DF.select('stanice', 'stat', 'nazev').distinct()
Tep_DF2.show()

### Transformace sloupců

In [ ]:
# vytvoreni noveho sloupce
Tep_DF2 = Tep_DF.withColumn('teplota_f', Tep_DF['teplota']*9.0/5.0 + 32)
Tep_DF2.show()

# prepsani existujiciho sloupce
Tep_DF2 = Tep_DF.withColumn('teplota', Tep_DF['teplota']*9.0/5.0 + 32)
Tep_DF2.show()

# pokud potrebuji aplikovat funkci na kazdy prvek sloupce: Spark SQL sloupcove (vektorove) funkce
# viz http://spark.apache.org/docs/1.6.0/api/python/pyspark.sql.html#module-pyspark.sql.functions
# je potreba importovat modul
# from pyspark.sql import functions as F
Tep_DF2 = Tep_DF.withColumn('stanice2', F.lower(Tep_DF['stanice']))
Tep_DF2.show()

Tep_DF2 = Tep_DF.withColumn('nazev_delka', F.length(Tep_DF['nazev']))
Tep_DF2.show()

Tep_DF2 = Tep_DF.withColumn('nazev_slov', F.size(F.split(Tep_DF['nazev'], " ")))
Tep_DF2.show()

Tep_DF2 = Tep_DF.withColumn('nazev2', F.regexp_replace(Tep_DF['nazev'], 'A', '4'))
Tep_DF2.show()

Tep_DF2 = Tep_DF.withColumn('den2', F.when(Tep_DF['den']<=10, '1-10').otherwise('11-31'))
Tep_DF2.show()

### prace s chybejicimi hodnotami
Tep_DF2 = Tep_DF.dropna() # vyhodi radky s aspon jednou hodnotou null
Tep_DF2.show()

Tep_DF2 = Tep_DF.fillna(0, 'teplota')
Tep_DF2.show()

### prejmenovani sloupcu
# jeden sloupec
Tep_DF2 = Tep_DF.withColumnRenamed('latitude', 'gps_lat')
Tep_DF2.show()

# vsechny sloupce v DataFrame
Tep_DF2 = Tep_DF.select('stat', 'latitude', 'longitude').toDF('usa_state', 'gps_lat', 'gps_long')
Tep_DF2.show()

### Řazení, agregace

In [ ]:
### razeni
Tep_DF2 = Tep_DF.orderBy('latitude', ascending=False)
Tep_DF2.show()

Tep_DF2 = Tep_DF.orderBy(Tep_DF['latitude'].desc())
Tep_DF2.show()

### agregace
Tep_DF2 = Tep_DF.groupBy('mesic').avg('teplota')
Tep_DF2.show()

Tep_DF2 = Tep_DF.groupBy('mesic').agg({'teplota': 'avg'}) # to same v obecnejsim zapisu
Tep_DF2.show()

Tep_DF2 = Tep_DF.groupBy('stat').count()
Tep_DF2.show()

# agregace pres cely DataFrame
Tep_DF2 = Tep_DF.max('teplota') # nebude fungovat, je treba udelat prazdny groupBy
Tep_DF2 = Tep_DF.groupBy().max('teplota') # takto funguje, vysledek je DataFrame
Tep_DF2.show()

### Join - spojování tabulek

In [ ]:
# druha tabulka - na pripojeni k vychozimu DataFrame
States = sqlContext.createDataFrame([('CA', 'California'), ('TX', 'Texas'), ('KY', 'Kentucky')], ('stat', 'stat_nazev'))
States.show()

# join pri shode jmen pole, pres ktere se joinuje
Tep_DF2 = Tep_DF.join(States, 'stat')
Tep_DF2.show()

# join s obecnejsi joinovaci podminkou - bude obsahovat joinovaci pole 2x
Tep_DF2 = Tep_DF.join(States, Tep_DF['stat']==States['stat'])
Tep_DF2.show()